# Segment Anything Model for Geospatial Data

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/satellite.ipynb)

This notebook shows how to use segment satellite imagery using the Segment Anything Model (SAM) with a few lines of code.

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator.

## Install dependencies

Uncomment and run the following cell to install the required dependencies.


In [7]:
 pip install segment-geospatial

## Import libraries

In [8]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

## Create an interactive map

In [9]:
m = leafmap.Map(center=[37.8713, -122.2580], zoom=19)
#m.add_basemap("SATELLITE")
m

Map(center=[37.8713, -122.258], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [12]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

In [11]:
m.user_roi_bounds()

[-122.267, 37.8685, -122.2524, 37.875]

## Download map tiles

Download maps tiles and mosaic them into a single GeoTIFF file

In [13]:
image = "satellite.tif"

Besides the `satellite` basemap, you can use any of the following basemaps returned by the `get_basemaps()` function:

In [14]:
# get_basemaps().keys()

dict_keys(['BaseMapDE.Color', 'BaseMapDE.Grey', 'BasemapAT.basemap', 'BasemapAT.grau', 'BasemapAT.highdpi', 'BasemapAT.orthofoto', 'BasemapAT.overlay', 'BasemapAT.surface', 'BasemapAT.terrain', 'CartoDB.DarkMatter', 'CartoDB.DarkMatterNoLabels', 'CartoDB.DarkMatterOnlyLabels', 'CartoDB.Positron', 'CartoDB.PositronNoLabels', 'CartoDB.PositronOnlyLabels', 'CartoDB.Voyager', 'CartoDB.VoyagerLabelsUnder', 'CartoDB.VoyagerNoLabels', 'CartoDB.VoyagerOnlyLabels', 'CyclOSM', 'Esri.AntarcticBasemap', 'Esri.AntarcticImagery', 'Esri.ArcticImagery', 'Esri.ArcticOceanBase', 'Esri.ArcticOceanReference', 'Esri.NatGeoWorldMap', 'Esri.OceanBasemap', 'Esri.WorldGrayCanvas', 'Esri.WorldImagery', 'Esri.WorldPhysical', 'Esri.WorldShadedRelief', 'Esri.WorldStreetMap', 'Esri.WorldTerrain', 'Esri.WorldTopoMap', 'FreeMapSK', 'Gaode.Normal', 'Gaode.Satellite', 'GeoportailFrance.500k', 'GeoportailFrance.Acces_Biomethane', 'GeoportailFrance.Adminexpress_cog_2017', 'GeoportailFrance.Adminexpress_cog_2018', 'Geopor

Specify the basemap as the source.

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 0001/1100
Downloaded image 0002/1100
Downloaded image 0003/1100
Downloaded image 0004/1100
Downloaded image 0005/1100
Downloaded image 0006/1100
Downloaded image 0007/1100
Downloaded image 0008/1100
Downloaded image 0009/1100
Downloaded image 0010/1100
Downloaded image 0011/1100
Downloaded image 0012/1100
Downloaded image 0013/1100
Downloaded image 0014/1100
Downloaded image 0015/1100
Downloaded image 0016/1100
Downloaded image 0017/1100
Downloaded image 0018/1100
Downloaded image 0019/1100
Downloaded image 0020/1100
Downloaded image 0021/1100
Downloaded image 0022/1100
Downloaded image 0023/1100
Downloaded image 0024/1100
Downloaded image 0025/1100
Downloaded image 0026/1100
Downloaded image 0027/1100
Downloaded image 0028/1100
Downloaded image 0029/1100
Downloaded image 0030/1100
Downloaded image 0031/1100
Downloaded image 0032/1100
Downloaded image 0033/1100
Downloaded image 0034/1100
Downloaded image 0035/1100
Downloaded image 0036/1100
Downloaded image 0037/1100
D

You can also use your own image. Uncomment and run the following cell to use your own image.

In [ ]:
# image = '/path/to/your/own/image.tif'

Display the downloaded image on the map.

In [ ]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

![](https://i.imgur.com/KAm84IY.png)

## Initialize SAM class

In [4]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

NameError: name 'SamGeo' is not defined

## Segment the image

Set `batch=True` to segment the image in batches. This is useful for large images that cannot fit in memory.

In [ ]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

## Polygonize the raster data

Save the segmentation results as a GeoPackage file.

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

You can also save the segmentation results as any vector data format supported by GeoPandas.

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

## Visualize the results

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m

![](https://i.imgur.com/Ysq3u7E.png)